In [1]:
import tensorflow as tf
import numpy as np
from scipy.misc import imread, imresize, imsave, imshow
import os

# valence
valence = np.arange(0.75, -0.751, -0.25)
valence = np.repeat(valence, 7).reshape((49, 1))

# arousal
arousal = [np.arange(0.75, -0.751, -0.25)]
arousal = np.repeat(arousal, 7, axis=0)
arousal = np.asarray([item for sublist in arousal for item in sublist]).reshape((49, 1))

def save_output(input_image, output, path, image_value_range = (-1,1), size_frame=[7, 10]):

    # Tile black background
    black_image = np.zeros((1, 96, 96, 3))
    black_image3 = np.tile(black_image, (3, 1, 1, 1))

    # Build final image from components
    input_frame = np.concatenate([black_image3,
                                  output[:7],
                                  black_image3,
                                  output[7:14],
                                  black_image3,
                                  output[14:21],
                                  black_image, input_image, black_image,
                                  output[21:28],
                                  black_image3,
                                  output[28:35],
                                  black_image3,
                                  output[35:42],
                                  black_image3,
                                  output[42:]
                                  ])

    # Transform into savable format
    final_image = get_images_frame(input_frame, image_value_range=image_value_range, size_frame=size_frame)

    # Save image
    imsave(path, final_image)

def show_output(input_image, output, path, image_value_range = (-1,1), size_frame=[7, 10]):

    # Tile black background
    black_image = np.zeros((1, 96, 96, 3))
    black_image3 = np.tile(black_image, (3, 1, 1, 1))

    # Build final image from components
    input_frame = np.concatenate([black_image3,
                                  output[:7],
                                  black_image3,
                                  output[7:14],
                                  black_image3,
                                  output[14:21],
                                  black_image, input_image, black_image,
                                  output[21:28],
                                  black_image3,
                                  output[28:35],
                                  black_image3,
                                  output[35:42],
                                  black_image3,
                                  output[42:]
                                  ])

    # Transform into savable format
    final_image = get_images_frame(input_frame, image_value_range=image_value_range, size_frame=size_frame)

    # Save image
    imshow(path, final_image)


def get_images_frame(
        batch_images,  # a batch of images
        image_value_range=(-1, 1),  # value range of the input batch images
        size_frame=None  # size of the image matrix, number of images in each row and column
):
    # transform the pixcel value to 0~1
    images = (batch_images - image_value_range[0]) / (image_value_range[-1] - image_value_range[0])
    if size_frame is None:
        auto_size = int(np.ceil(np.sqrt(images.shape[0])))
        size_frame = [auto_size, auto_size]
    img_h, img_w = batch_images.shape[1], batch_images.shape[2]
    frame = np.zeros([img_h * size_frame[0], img_w * size_frame[1], 3])
    for ind, image in enumerate(images):
        ind_col = ind % size_frame[1]
        ind_row = ind // size_frame[1]
        frame[(ind_row * img_h):(ind_row * img_h + img_h), (ind_col * img_w):(ind_col * img_w + img_w), :] = image
    return frame

def load_image(
        image_path,  # path of a image
        image_size=96,  # expected size of the image
        image_value_range=(-1, 1),  # expected pixel value range of the image
        is_gray=False,  # gray scale or color image
):
    if is_gray:
        image = imread(image_path, flatten=True).astype(np.float32)
    else:
        image = imread(image_path).astype(np.float32)
    image = imresize(image, [image_size, image_size])
    image = image.astype(np.float32) * (image_value_range[-1] - image_value_range[0]) / 255.0 + image_value_range[0]
    return image


/home/student/tensorflow_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # restore graph 
    new_saver = tf.train.import_meta_graph('checkpoint_00/02_model.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('./checkpoint_00'))
    graph = tf.get_default_graph()
    
    # create feed dict
    arousal_tensor = graph.get_tensor_by_name("arousal_labels:0")
    valence_tensor = graph.get_tensor_by_name("valence_labels:0")
    images_tensor = graph.get_tensor_by_name("input_images:0")
    
    # load input
    path = './selected_celebs/'
    save_path = './selected_celebs_out/00/'
    files_already = os.listdir(save_path)
    
    files = [f for f in os.listdir(path) if not f in files_already]
    print(len(files))
    
    for file in files:
        i = load_image(path+file).reshape((1,96,96,3)) #np.ones((1,96,96,3)).astype(np.float32)*-0. # #
        query_images = np.tile(i, (49, 1, 1, 1))

        # create input for net
        feed_dict ={arousal_tensor:arousal,valence_tensor:valence, images_tensor:query_images}
        op_to_restore = sess.graph.get_tensor_by_name("generator/Tanh:0")

        # run
        x = sess.run(op_to_restore,feed_dict)

        # save
        save_output(i, x, save_path+file)

    





INFO:tensorflow:Restoring parameters from ./checkpoint_00/02_model
54


/home/student/tensorflow_env/lib/python3.5/site-packages/ipykernel_launcher.py:101: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/student/tensorflow_env/lib/python3.5/site-packages/ipykernel_launcher.py:102: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/student/tensorflow_env/lib/python3.5/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [36]:
changes = np.absolute(neutral-surprise)



In [ ]:
import sys
import numpy
from PIL import Image

result = Image.fromarray((np.ones((100,100,3))*255).astype(numpy.uint8))
result.save('out.png')

In [ ]:
i = load_image("light_skin.png", image_value_range=(0, 255))
pyplot.imshow(i)

In [ ]:
import matplotlib.pyplot as pyplot


In [91]:
path = './cutted_443/'
files = [path+i for i in os.listdir(path)]






/home/student/tensorflow_env/lib/python3.5/site-packages/ipykernel_launcher.py:26: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [ ]:
import pylab
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

img1 = mpimg.imread('./cutted_443/01sXs0s0.png')
print(img1.shape)
img2 = mpimg.imread('./cutted_443/01sXs0s.png')
print(img2.shape)

# Calculate the absolute difference on each channel separately
error_r = np.fabs(np.subtract(img2[:,:,0], img1[:,:,0]))
error_g = np.fabs(np.subtract(img2[:,:,1], img1[:,:,1]))
error_b = np.fabs(np.subtract(img2[:,:,2], img1[:,:,2]))

# Calculate the maximum error for each pixel
lum_img = np.maximum(np.maximum(error_r, error_g), error_b)

# Uncomment the next line to turn the colors upside-down
#lum_img = np.negative(lum_img);

imgplot = plt.imshow(lum_img)

# Choose a color palette
imgplot.set_cmap('gist_gray')
#imgplot.set_cmap('Spectral') 

plt.colorbar()
plt.axis('off')

pylab.show()